## Lab 3_2 - Use Resnet (pretrained) on Kaggle imagenet-256 data set

In [1]:
import os
import torch
import torchvision.transforms as transforms
from torchvision import datasets, models
from torch.utils.data import DataLoader

In [2]:
# Define constants
BATCH_SIZE = 256
NUM_WORKERS = 4

In [3]:

# Data transformations
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [4]:
!pip install kagglehub
import kagglehub
from torchvision.datasets import ImageFolder


# Load dataset
dataset_path = kagglehub.dataset_download("dimensi0n/imagenet-256")
dataset = ImageFolder(root=dataset_path, transform=transform)

print(f"Total images in the dataset: {len(dataset)}")  # Print the total number of images
total_classes = len(dataset.classes)
print("Total number of classes:", total_classes)


100%|██████████| 7.15G/7.15G [05:28<00:00, 23.4MB/s]

Extracting files...


Total images in the dataset: 539826
Total number of classes: 1000


In [5]:
# Step 2: Load the dataset
# test_dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
test_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
from torchvision import models
model = models.resnet18(pretrained=True).to(device)
print(model)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 193MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [8]:
from torch.nn.functional import softmax

# Step 4: Compute accuracy with better mapping
correct = 0
total = 0
processed_images = 0  # Counter for tracking progress

c = 0

with torch.no_grad():
    for images, labels in test_loader:
      images = images.to(device); labels = labels.to(device) # Move tensors to cuda
      outputs = model(images)
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
        # Update the number of processed images
      processed_images += labels.size(0)
        # Print progress every 1000 images
      if processed_images % 1000 == 0:
          c = c+1
          accuracy = 100 * correct / total
          print(f"Processed {processed_images} images so far... and accuaracy is {accuracy:.2f}%")

# Compute and print final accuracies
accuracy = 100 * correct / total
print(f"Final Accuracy: {accuracy:.2f}%")


Processed 32000 images so far... and accuaracy is 0.08%
Processed 64000 images so far... and accuaracy is 0.08%
Processed 96000 images so far... and accuaracy is 0.08%
Processed 128000 images so far... and accuaracy is 0.07%
Processed 160000 images so far... and accuaracy is 0.08%
Processed 192000 images so far... and accuaracy is 0.08%
Processed 224000 images so far... and accuaracy is 0.09%
Processed 256000 images so far... and accuaracy is 0.09%
Processed 288000 images so far... and accuaracy is 0.09%
Processed 320000 images so far... and accuaracy is 0.09%
Processed 352000 images so far... and accuaracy is 0.10%
Processed 384000 images so far... and accuaracy is 0.10%
Processed 416000 images so far... and accuaracy is 0.10%
Processed 448000 images so far... and accuaracy is 0.10%
Processed 480000 images so far... and accuaracy is 0.11%
Processed 512000 images so far... and accuaracy is 0.10%
Final Accuracy: 0.10%
